In [1]:
!pip install geopandas
!pip install mapclassify
!pip install contextily

!pip install -U kaleido
!pip install -U plotly
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.7/286.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 kB 4.4 MB/s eta 0:00:00


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 56.4 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.24.1
    Uninstalling plotly-5.24.1:
      Successfully uninstalled plotly-5.24.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 34.2 MB/s eta 0:00:00


In [2]:
import pandas as pd
import geopandas as gp
import numpy as np

import folium

from branca import colormap
from branca.element import MacroElement

from jinja2 import Template
import plotly.express as px

import os
import io

import kaleido
from PIL import Image
import selenium

from google.colab import files
from google.colab import drive

In [ ]:
!sudo apt-get update
!sudo apt-get install google-chrome-stable

In [ ]:
chrome_path = str(kaleido.get_chrome_sync())
os.environ['KALEIDO_CHROME_PATH'] = chrome_path

In [3]:
#configuracion de pandas
pd.set_option('future.no_silent_downcasting', True)

In [4]:
# Preparing files
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# importando datasets
dataset_root = '/content/drive/MyDrive/Analisis de Datos 2024 - 2025/Final - Lautaro Wojcik/datasets/'

In [ ]:
###############################################################################################################

In [6]:
# Alumnos, nuevos inscriptos, reinscriptos y egresados de carreras de pregrado y grado en instituciones universitarias, según jurisdicción.
# Total del país. Año 2021 y egresados de 2020
df_provincias_gestion_privada = pd.read_excel(dataset_root + '030410_2022.xlsx', header=[3, 4], nrows=19)
df_provincias_gestion_publica = pd.read_excel(dataset_root + '030409_2022.xlsx', header=[2, 3], nrows=25)

In [7]:
# Alumnos, nuevos inscriptos, reinscriptos y egresados de carreras de pregrado y grado, según instituciones universitarias.
# Total del país. Año 2021 y egresados de 2020
df_universidades_gestion_publica = pd.read_excel(dataset_root + '030407_2022.xlsx', header=[2, 3], nrows=72)
df_universidades_gestion_privada = pd.read_excel(dataset_root + '030408_2022.xlsx', header=[3, 4], nrows=65)

In [8]:
# Universidades * Provincias + Data (hecho manualmente) con datos formalizados
df_universidades_complementario = pd.read_csv(dataset_root + 'Instituciones de Formación Superior.csv')

In [9]:
# Alumnos repitentes de educación común, por nivel de enseñanza y sector de gestión, según jurisdicción. Total del país. Año 2022
df_repitentes = pd.read_excel(dataset_root + '030402_2022.xlsx', header=[2, 3, 4, 5, 6], skiprows=1, nrows=27)

In [10]:
# Alumnos de educación común, por nivel de enseñanza y sector de gestión, según jurisdicción. Total del país. Año 2022
df_alumnos = pd.read_excel(dataset_root + '030401_2022.xlsx', header=[1, 2, 3, 4, 5], skiprows=2, nrows=27)

In [11]:
# Provincias geo-data
gdf = gp.read_file(dataset_root + 'ProvinciasArgentina.geojson')

In [12]:
# normalizacion de encabezado
def normalize_headers(df):
  renamed_columns = []
  for column in df.columns:
    normalized_column = ''
    if isinstance(column, str):
      normalized_column = column

    else:
      for column_header in column:
        column_header = str(column_header)
        if 'Unnamed:' in column_header or column_header == '' or column_header == ' ':
          continue
        else:
          normalized_column += column_header + '_'

      if not normalized_column:
        normalized_column = 'empty'

    renamed_columns.append(normalized_column.replace(' ', '_').rstrip('_'))

  df.columns = renamed_columns
  return df

In [13]:
# Normalizando tablas de estudiantes de universidades
df_universidades_gestion_publica = normalize_headers(df_universidades_gestion_publica)
df_universidades_gestion_privada = normalize_headers(df_universidades_gestion_privada)

df_universidades_complementario = normalize_headers(df_universidades_complementario)

# concatenando tablas
df_universidades = pd.concat([df_universidades_gestion_publica, df_universidades_gestion_privada], ignore_index=True, sort=False)

# obteniendo subtotales

# Total de instituciones estatales
tot_inst_est = df_universidades[df_universidades['Institución'] == 'Total de instituciones estatales']

# Total de instituciones privadas
tot_inst_priv = df_universidades[df_universidades['Institución'] == 'Total de instituciones privadas']

# Total de universidades provinciales
tot_univ_prov = df_universidades[df_universidades['Institución'] == 'Total de universidades provinciales']

# Total de universidades nacionales
tot_univ_naci = df_universidades[df_universidades['Institución'] == 'Total de universidades nacionales']

# Total de universidades privadas
tot_univ_priv = df_universidades[df_universidades['Institución'] == 'Total de universidades privadas']

# Total de institutos universitarios provinciales
tot_inst_univ_prov = df_universidades[df_universidades['Institución'] == 'Total de institutos universitarios provinciales']

# Total de institutos universitarios nacionales
tot_inst_univ_naci = df_universidades[df_universidades['Institución'] == 'Total de institutos universitarios nacionales']

# Total de institutos universitarios privados
tot_inst_univ_priv = df_universidades[df_universidades['Institución'] == 'Total de institutos universitarios privados']

# removiendo subtotales
subtotales = [
    'Total de universidades provinciales',
    'Total de universidades nacionales',
    'Total de universidades privadas',
    'Total de institutos universitarios provinciales',
    'Total de institutos universitarios nacionales',
    'Total de institutos universitarios privados',
    'Total de instituciones estatales',
    'Total de instituciones privadas'
]

df_universidades_sin_subtotales = df_universidades[~df_universidades['Institución'].isin(subtotales)]

# normalizando nombre de provincia
df_universidades_complementario['Jurisdicción'] = df_universidades_complementario['Jurisdicción'].replace(
    'Tierra del Fuego', 'Tierra del Fuego, Antártida e Islas del Atlántico Sur'
)

# relacionando jurisdicciónes
df_universidades_con_complemento = df_universidades_sin_subtotales.merge(df_universidades_complementario, how='left', on='Institución')

# hay algunos datos que: '///' Dato que no corresponde presentar. o '...' / '…' Cero absoluto, los voy a convertir a 0
df_universidades_con_complemento['2021_Alumnos'] = df_universidades_con_complemento['2021_Alumnos'].replace('///', 0)
df_universidades_con_complemento['2021_Alumnos'] = df_universidades_con_complemento['2021_Alumnos'].replace('...', 0)
df_universidades_con_complemento['2021_Alumnos'] = df_universidades_con_complemento['2021_Alumnos'].replace('…', 0)
df_universidades_con_complemento['2021_Alumnos'] = df_universidades_con_complemento['2021_Alumnos'].astype(int)

df_universidades_con_complemento['2020_Egresados'] = df_universidades_con_complemento['2020_Egresados'].replace('///', 0)
df_universidades_con_complemento['2020_Egresados'] = df_universidades_con_complemento['2020_Egresados'].replace('...', 0)
df_universidades_con_complemento['2020_Egresados'] = df_universidades_con_complemento['2020_Egresados'].replace('…', 0)
df_universidades_con_complemento['2020_Egresados'] = df_universidades_con_complemento['2020_Egresados'].astype(int)

df_universidades_con_complemento['2021_Nuevos_inscriptos'] = df_universidades_con_complemento['2021_Nuevos_inscriptos'].replace('///', 0)
df_universidades_con_complemento['2021_Nuevos_inscriptos'] = df_universidades_con_complemento['2021_Nuevos_inscriptos'].replace('...', 0)
df_universidades_con_complemento['2021_Nuevos_inscriptos'] = df_universidades_con_complemento['2021_Nuevos_inscriptos'].replace('…', 0)
df_universidades_con_complemento['2021_Nuevos_inscriptos'] = df_universidades_con_complemento['2021_Nuevos_inscriptos'].astype(int)

df_universidades_con_complemento['2021_Reinscriptos'] = df_universidades_con_complemento['2021_Reinscriptos'].replace('///', 0)
df_universidades_con_complemento['2021_Reinscriptos'] = df_universidades_con_complemento['2021_Reinscriptos'].replace('...', 0)
df_universidades_con_complemento['2021_Reinscriptos'] = df_universidades_con_complemento['2021_Reinscriptos'].replace('…', 0)
df_universidades_con_complemento['2021_Reinscriptos'] = df_universidades_con_complemento['2021_Reinscriptos'].astype(int)

df_universidades_con_complemento['2021_Egresados'] = df_universidades_con_complemento['2021_Egresados'].replace('///', 0)
df_universidades_con_complemento['2021_Egresados'] = df_universidades_con_complemento['2021_Egresados'].replace('...', 0)
df_universidades_con_complemento['2021_Egresados'] = df_universidades_con_complemento['2021_Egresados'].replace('…', 0)
df_universidades_con_complemento['2021_Egresados'] = df_universidades_con_complemento['2021_Egresados'].astype(int)

In [15]:
# Normalizando tablas de estudiantes de provincias
df_provincias_gestion_publica = normalize_headers(df_provincias_gestion_publica)
df_provincias_gestion_privada = normalize_headers(df_provincias_gestion_privada)

# combinando tablas
df_provincias = df_provincias_gestion_privada.merge(df_provincias_gestion_publica, how='outer', on='Jurisdicción', suffixes=('_privada', '_publica'))

# obteniendo subtotales

# Total general
tot_gnrl = df_provincias[df_provincias['Jurisdicción'] == 'Total general']

# removiendo subtotales
subtotales = [
    'Total general'
]

df_provincias_sin_subtotales = df_provincias[~df_provincias['Jurisdicción'].isin(subtotales)]

In [17]:
# Normalizando tablas de repitentes de provincias
df_repitentes = normalize_headers(df_repitentes)

# renombrando primera columna
df_repitentes.columns.values[0] = "Jurisdicción"

df_repitentes.columns = [
    'Jurisdicción',
    '2022_Primario_Total',
    '2022_Primario_Sector_Estatal',
    '2022_Primario_Sector_Privado',
    '2022_Primario_Sector_Privado_Duplicado',
    '2022_Secundario_Total',
    '2022_Secundario_Sector_Estatal',
    '2022_Secundario_Sector_Privado']

# descartando columna repetida
df_repitentes_sin_columnas_duplicdas = df_repitentes.drop(df_repitentes.columns[4], axis=1)

# corrigiendo indices
df_repitentes_sin_columnas_duplicdas['Jurisdicción'] = df_repitentes_sin_columnas_duplicdas['Jurisdicción'].replace(
    'Ciudad de Buenos Aires', 'Ciudad Autónoma de Buenos Aires'
)

# obteniendo subtotales

# Total del país
tot_repitentes_pais = df_repitentes_sin_columnas_duplicdas[df_repitentes_sin_columnas_duplicdas['Jurisdicción'] == 'Total del país']

# removiendo subtotales
subtotales = [
    'Total del país',
    '24 partidos del Gran Buenos Aires',
    'Interior de la provincia de Buenos Aires'
]

df_repitentes_sin_subtotales = df_repitentes_sin_columnas_duplicdas[~df_repitentes_sin_columnas_duplicdas['Jurisdicción'].isin(subtotales)]

In [18]:
display(df_repitentes_sin_subtotales)

,Jurisdicción,2022_Primario_Total,2022_Primario_Sector_Estatal,2022_Primario_Sector_Privado,2022_Secundario_Total,2022_Secundario_Sector_Estatal,2022_Secundario_Sector_Privado
1,Ciudad Autónoma de Buenos Aires,3976,3525,451,8861,7422,1439
2,Buenos Aires,47500,45022,2478,87814,81954,5860
5,Catamarca,1021,1016,5,2922,2815,107
6,Chaco,2174,2067,107,8041,7472,569
7,Chubut,383,320,63,4771,4650,121
8,Córdoba,2067,2004,63,16784,14384,2400
9,Corrientes,8128,7957,171,5587,5126,461
10,Entre Ríos,6575,5974,601,9145,8555,590
11,Formosa,1389,1350,39,5554,5450,104
12,Jujuy,377,369,8,3784,3512,272


In [19]:
# Normalizando tablas de estudiantes de provincias por gestion
df_alumnos = normalize_headers(df_alumnos)

# renombrando primera columna
df_alumnos.columns.values[0] = "Jurisdicción"

# Total del país
tot_alumnos_pais = df_alumnos[df_alumnos['Jurisdicción'] == 'Total del país']

# removiendo subtotales
subtotales = [
    'Total del país',
    '24 partidos del Gran Buenos Aires',
    'Interior de la provincia de Buenos Aires'
]

df_alumnos_sin_subtotales = df_alumnos[~df_alumnos['Jurisdicción'].isin(subtotales)]

In [20]:
display(df_alumnos_sin_subtotales)

,Jurisdicción,2022_Total_Total,2022_Total_Sector_Estatal,2022_Total_Sector_Privado,2022_Inicial_Total,2022_Inicial_Sector_Estatal,2022_Inicial_Sector_Privado,2022_Primario__Total,2022_Primario__Sector_Estatal,2022_Primario__Sector_Privado,2022_Secundario_Total,2022_Secundario_Sector_Estatal,2022_Secundario_Sector_Privado,2022_Superior_no_universitario_Total,2022_Superior_no_universitario_Sector_Estatal,2022_Superior_no_universitario_Sector_Privado
1,Ciudad Autónoma de Buenos Aires,712626,353504,359122,105905,51096,54809,278233,145529,132704,203599,103233,100366,124889,53646,71243
2,Buenos Aires,4346278,2982721,1363557,672694,441583,231111,1734718,1164218,570500,1677473,1165688,511785,261393,211232,50161
5,Catamarca,115425,92151,23274,15693,11737,3956,42657,33758,8899,42702,34326,8376,14373,12330,2043
6,Chaco,385273,314992,70281,51422,42277,9145,170078,147758,22320,102739,83933,18806,61034,41024,20010
7,Chubut,156398,131714,24684,25166,20437,4729,60239,50882,9357,60640,51833,8807,10353,8562,1791
8,Córdoba,925392,621985,303407,136445,99916,36529,357084,259279,97805,347517,210432,137085,84346,52358,31988
9,Corrientes,326720,277115,49605,44214,36018,8196,130195,113372,16823,108826,90399,18427,43485,37326,6159
10,Entre Ríos,348968,254539,94429,51527,35676,15851,142480,103152,39328,129903,97356,32547,25058,18355,6703
11,Formosa,170228,149945,20283,20281,15450,4831,71956,64976,6980,62873,56508,6365,15118,13011,2107
12,Jujuy,219223,181101,38122,29022,22511,6511,93931,81936,11995,68022,57331,10691,28248,19323,8925


In [21]:
# Normalizando nombres de provincia
gdf.replace('Tierra del Fuego', 'Tierra del Fuego, Antártida e Islas del Atlántico Sur', inplace=True)
gdf.replace('Capital Federal', 'Ciudad Autónoma de Buenos Aires', inplace=True)

In [22]:
# fin de normalizacion, tablas resultantes:
#df_alumnos_sin_subtotales, df_repitentes_sin_subtotales, df_provincias_sin_subtotales, df_universidades_con_complemento, gdf

In [23]:
# combinando la data en un solo dataframe excepto el de instituciones (df_universidades_con_complemento) me parece que no tiene sentido

# obteniendo las jurisdicciones analizadas
juriciones = pd.concat([
    df_alumnos_sin_subtotales['Jurisdicción'],
    df_repitentes_sin_subtotales['Jurisdicción'],
    df_provincias_sin_subtotales['Jurisdicción'],
    gdf['nombre']
], ignore_index=True, sort=False)

df = pd.DataFrame({'Jurisdicción': juriciones.unique()})

df_provincias_sin_subtotales = df_provincias_sin_subtotales.fillna(0)

# Convert object columns to numeric, coercing errors and filling NaN
for col in ['2020_Egresados_privada', '2021_Egresados_privada', '2021_Alumnos_privada', '2021_Nuevos_inscriptos_privada', '2021_Reinscriptos_privada']:
  df_provincias_sin_subtotales[col] = pd.to_numeric(df_provincias_sin_subtotales[col], errors='coerce').fillna(0)

df = df.merge(df_provincias_sin_subtotales.add_suffix('_Universitarios'), how='left', left_on='Jurisdicción', right_on='Jurisdicción_Universitarios')

df_alumnos_sin_subtotales.rename(columns={'2022_Total_Total': '2022_Total_Total_No_Universitarios'})
df = df.merge(df_alumnos_sin_subtotales.add_suffix('_No_Universitarios'), how='left', left_on='Jurisdicción', right_on='Jurisdicción_No_Universitarios')

df = df.merge(df_repitentes_sin_subtotales.add_suffix('_Repitentes'), how='left', left_on='Jurisdicción', right_on='Jurisdicción_Repitentes')

df = df.drop(['Jurisdicción_Universitarios', 'Jurisdicción_No_Universitarios', 'Jurisdicción_Repitentes'], axis=1)

df = df.rename(columns={
    'Jurisdicción'                                                      : 'Jurisdicción',
    '2020_Egresados_privada_Universitarios'                             : '2020_Egresados_Privado_Universitarios',
    '2021_Alumnos_privada_Universitarios'                               : '2021_Privado_Universitarios',
    '2021_Nuevos_inscriptos_privada_Universitarios'                     : '2021_Inscriptos_Privado_Universitarios',
    '2021_Reinscriptos_privada_Universitarios'                          : '2021_Reinscriptos_Privado_Universitarios',
    '2021_Egresados_privada_Universitarios'                             : '2021_Egresados_Privado_Universitarios',
    '2020_Egresados_publica_Universitarios'                             : '2020_Egresados_Estatal_Universitarios',
    '2021_Alumnos_publica_Universitarios'                               : '2021_Estatal_Universitarios',
    '2021_Nuevos_inscriptos_publica_Universitarios'                     : '2021_Inscriptos_Estatal_Universitarios',
    '2021_Reinscriptos_publica_Universitarios'                          : '2021_Reinscriptos_Estatal_Universitarios',
    '2021_Egresados_publica_Universitarios'                             : '2021_Egresados_Estatal_Universitarios',
    '2022_Total_Total_No_Universitarios'                                : '2022_Total_No_Universitarios',
    '2022_Total_Sector_Estatal_No_Universitarios'                       : '2022_Total_Estatal_No_Universitarios',
    '2022_Total_Sector_Privado_No_Universitarios'                       : '2022_Total_Privado_No_Universitarios',
    '2022_Inicial_Total_No_Universitarios'                              : '2022_Total_Inicial',
    '2022_Inicial_Sector_Estatal_No_Universitarios'                     : '2022_Estatal_Inicial',
    '2022_Inicial_Sector_Privado_No_Universitarios'                     : '2022_Privado_Inicial',
    '2022_Primario__Total_No_Universitarios'                            : '2022_Total_Primario',
    '2022_Primario__Sector_Estatal_No_Universitarios'                   : '2022_Estatal_Primario',
    '2022_Primario__Sector_Privado_No_Universitarios'                   : '2022_Privado_Primario',
    '2022_Secundario_Total_No_Universitarios'                           : '2022_Total_Secundario',
    '2022_Secundario_Sector_Estatal_No_Universitarios'                  : '2022_Estatal_Secundario',
    '2022_Secundario_Sector_Privado_No_Universitarios'                  : '2022_Privado_Secundario',
    '2022_Superior_no_universitario_Total_No_Universitarios'            : '2022_Total_Superior',
    '2022_Superior_no_universitario_Sector_Estatal_No_Universitarios'   : '2022_Estatal_Superior',
    '2022_Superior_no_universitario_Sector_Privado_No_Universitarios'   : '2022_Privado_Superior',
    '2022_Primario_Total_Repitentes'                                    : '2022_Total_Repitentes_Primario',
    '2022_Primario_Sector_Estatal_Repitentes'                           : '2022_Estatal_Repitentes_Primario',
    '2022_Primario_Sector_Privado_Repitentes'                           : '2022_Privado_Repitentes_Primario',
    '2022_Secundario_Total_Repitentes'                                  : '2022_Total_Repitentes_Secundario',
    '2022_Secundario_Sector_Estatal_Repitentes'                         : '2022_Estatal_Repitentes_Secundario',
    '2022_Secundario_Sector_Privado_Repitentes'                         : '2022_Privado_Repitentes_Secundario'
})

In [24]:
#------------------------------------2021_Total_Universitarios

df['2021_Total_Universitarios'] = df['2021_Estatal_Universitarios'] + df['2021_Privado_Universitarios']
df['2021_Total_Universitarios'] = df['2021_Total_Universitarios'].astype(int)

#------------------------------------2021_Porcentaje_Universitarios, 2021_Porcentaje_Estatal_Universitarios, 2021_Porcentaje_Privado_Universitarios
  #% arg
df['2021_Porcentaje_Universitarios'] = (df['2021_Total_Universitarios']  * 100) / df['2021_Total_Universitarios'].sum()
df['2021_Porcentaje_Universitarios'] = df['2021_Porcentaje_Universitarios'].round(2).astype(str) + '%'

  #% per province
df['2021_Porcentaje_Estatal_Universitarios'] = (df['2021_Estatal_Universitarios']  * 100) / df['2021_Total_Universitarios']
df['2021_Porcentaje_Estatal_Universitarios'] = df['2021_Porcentaje_Estatal_Universitarios'].round(2).astype(str) + '%'

df['2021_Porcentaje_Privado_Universitarios'] = (df['2021_Privado_Universitarios']  * 100) / df['2021_Total_Universitarios']
df['2021_Porcentaje_Privado_Universitarios'] = df['2021_Porcentaje_Privado_Universitarios'].round(2).astype(str) + '%'

#------------------------------------2022_Porcentaje_Superior, 2022_Porcentaje_Estatal_Superior, 2022_Porcentaje_Privado_Superior
  #% arg
df['2022_Porcentaje_Superior'] = (df['2022_Total_Superior']  * 100) / df['2022_Total_Superior'].sum()
df['2022_Porcentaje_Superior'] = df['2022_Porcentaje_Superior'].round(2).astype(str) + '%'

  #% per province
df['2022_Porcentaje_Estatal_Superior'] = (df['2022_Estatal_Superior']  * 100) / df['2022_Total_Superior']
df['2022_Porcentaje_Estatal_Superior'] = df['2022_Porcentaje_Estatal_Superior'].round(2).astype(str) + '%'

df['2022_Porcentaje_Privado_Superior'] = (df['2022_Privado_Superior']  * 100) / df['2022_Total_Superior']
df['2022_Porcentaje_Privado_Superior'] = df['2022_Porcentaje_Privado_Superior'].round(2).astype(str) + '%'

#------------------------------------2022_Porcentaje_Secundario, 2022_Porcentaje_Estatal_Secundario, 2022_Porcentaje_Privado_Secundario
  #% arg
df['2022_Porcentaje_Secundario'] = (df['2022_Total_Secundario']  * 100) / df['2022_Total_Secundario'].sum()
df['2022_Porcentaje_Secundario'] = df['2022_Porcentaje_Secundario'].round(2).astype(str) + '%'

  #% per province
df['2022_Porcentaje_Estatal_Secundario'] = (df['2022_Estatal_Secundario']  * 100) / df['2022_Total_Secundario']
df['2022_Porcentaje_Estatal_Secundario'] = df['2022_Porcentaje_Estatal_Secundario'].round(2).astype(str) + '%'

df['2022_Porcentaje_Privado_Secundario'] = (df['2022_Privado_Secundario']  * 100) / df['2022_Total_Secundario']
df['2022_Porcentaje_Privado_Secundario'] = df['2022_Porcentaje_Privado_Secundario'].round(2).astype(str) + '%'

#------------------------------------2022_Porcentaje_Primario, 2022_Porcentaje_Estatal_Primario, 2022_Porcentaje_Privado_Primario
  #% arg
df['2022_Porcentaje_Primario'] = (df['2022_Total_Primario']  * 100) / df['2022_Total_Primario'].sum()
df['2022_Porcentaje_Primario'] = df['2022_Porcentaje_Primario'].round(2).astype(str) + '%'

  #% per province
df['2022_Porcentaje_Estatal_Primario'] = (df['2022_Estatal_Primario']  * 100) / df['2022_Total_Primario']
df['2022_Porcentaje_Estatal_Primario'] = df['2022_Porcentaje_Estatal_Primario'].round(2).astype(str) + '%'

df['2022_Porcentaje_Privado_Primario'] = (df['2022_Privado_Primario']  * 100) / df['2022_Total_Primario']
df['2022_Porcentaje_Privado_Primario'] = df['2022_Porcentaje_Privado_Primario'].round(2).astype(str) + '%'

#------------------------------------2022_Porcentaje_Inicial, 2022_Porcentaje_Estatal_Inicial, 2022_Porcentaje_Privado_Inicial
  #% arg
df['2022_Porcentaje_Inicial'] = (df['2022_Total_Inicial']  * 100) / df['2022_Total_Inicial'].sum()
df['2022_Porcentaje_Inicial'] = df['2022_Porcentaje_Inicial'].round(2).astype(str) + '%'

  #% per province
df['2022_Porcentaje_Estatal_Inicial'] = (df['2022_Estatal_Inicial']  * 100) / df['2022_Total_Inicial']
df['2022_Porcentaje_Estatal_Inicial'] = df['2022_Porcentaje_Estatal_Inicial'].round(2).astype(str) + '%'

df['2022_Porcentaje_Privado_Inicial'] = (df['2022_Privado_Inicial']  * 100) / df['2022_Total_Inicial']
df['2022_Porcentaje_Privado_Inicial'] = df['2022_Porcentaje_Privado_Inicial'].round(2).astype(str) + '%'

#------------------------------------2022_Porcentaje_Repitentes_Secundario, 2022_Porcentaje_Repitentes_Primario
  #%per province
df['2022_Porcentaje_Repitentes_Secundario'] = (df['2022_Total_Repitentes_Secundario']  * 100) / df['2022_Total_Secundario']
df['2022_Porcentaje_Repitentes_Secundario'] = df['2022_Porcentaje_Repitentes_Secundario'].round(2).astype(str) + '%'

df['2022_Porcentaje_Repitentes_Primario'] = (df['2022_Total_Repitentes_Primario']  * 100) / df['2022_Total_Primario']
df['2022_Porcentaje_Repitentes_Primario'] = df['2022_Porcentaje_Repitentes_Primario'].round(2).astype(str) + '%'

In [26]:
# siguiente: graficos, tablas:
# df_universidades_con_complemento, df, gdf

In [27]:
# que graficos hacer? req = +1general +1mapa
# n° de estudiantes: nivel educativo * estatal/privado * provincia - grafico + mapa?: mapa
# n° de repitentes: nivel educativo * provincia - grafico + mapa?: --^
# n° de estudiantes * institucion * estatal/privado  * provincia - grafico + mapa?: grafico

In [28]:
#Add branca.colormap.LinearColormap to an folium.FeatureGroup?
#https://stackoverflow.com/questions/72393558/add-branca-colormap-linearcolormap-to-an-folium-featuregroup
class BindColormap(MacroElement):
    """Binds a colormap to a given layer.

    Parameters
    ----------
    colormap : branca.colormap.ColorMap
        The colormap to bind.
    """
    def __init__(self, layer, colormap):
        super(BindColormap, self).__init__()
        self.layer = layer
        self.colormap = colormap
        self._template = Template(u"""
        {% macro script(this, kwargs) %}
            function updateColormapVisibility(colormap_, layer_) {
                if ({{this._parent.get_name()}}.hasLayer(layer_)) {
                    colormap_.svg[0][0].style.display = 'block';
                } else {
                    colormap_.svg[0][0].style.display = 'none';
                }
            }

            updateColormapVisibility({{this.colormap.get_name()}}, {{this.layer.get_name()}})
            {{this._parent.get_name()}}.on('overlayadd', function (eventLayer) {
                if (eventLayer.layer == {{this.layer.get_name()}}) {
                    updateColormapVisibility({{this.colormap.get_name()}}, {{this.layer.get_name()}})
                }});
            {{this._parent.get_name()}}.on('overlayremove', function (eventLayer) {
                if (eventLayer.layer == {{this.layer.get_name()}}) {
                    updateColormapVisibility({{this.colormap.get_name()}}, {{this.layer.get_name()}})
                }});
        {% endmacro %}
        """)  # noqa

In [29]:
def makeLayer(gdf, title, mainCol, tooltipData, show_ = False):
  layer = folium.FeatureGroup(name=title, show=show_)
  min = gdf[mainCol].quantile(0)
  max = gdf[mainCol].quantile(1)

  legend1 = colormap.linear.GnBu_09.scale(min, max)
  legend1.caption = title

  legend2 = colormap.linear.GnBu_08.scale(min, max)

  fg = folium.GeoJson(
    data = gdf,
    highlight_function = lambda feature: {
      'fillColor': legend1(feature['properties'][mainCol]),
      'color': 'black',
      'weight': 1,
      'fillOpacity': 0.9
    },
    style_function = lambda feature: {
      'fillColor': legend2(feature['properties'][mainCol]),
      'color': 'black',
      'weight': 2,
      'fillOpacity': 1.0
    }
  )

  folium.GeoJsonTooltip(
    fields = tooltipData['fields'],
    aliases = tooltipData['aliases'],
    localize = True,
    sticky = True
  ).add_to(fg)

  fg.add_to(layer)

  return layer, legend1

In [52]:
gdf_merged = gdf.merge(df, how='left', left_on='nombre', right_on='Jurisdicción')

Argentina = [-34.603722, -58.381592]
map = folium.Map(location=Argentina, zoom_start = 4)

map.add_child(folium.TileLayer('CartoDB Positron'))
#map.add_child(folium.TileLayer('CartoDB dark_matter'))

#------------------------------------Estudiantes de Nivel Universitario por Provincia 2021

layer, legend = makeLayer(gdf_merged, 'Estudiantes de Nivel Universitario por Provincia 2021', '2021_Total_Universitarios', {
    'fields': ['nombre', '2021_Total_Universitarios', '2021_Porcentaje_Universitarios', '2021_Porcentaje_Estatal_Universitarios', '2021_Porcentaje_Privado_Universitarios'],
    'aliases': ['Provincia:', 'Numero de Estudiantes:', 'Porcentaje de Estudiantes Respecto al Pais:', 'Porcentaje de Estudiantes de Estatales y Provinciales:', 'Porcentaje de Estudiantes de Privadas:']
})

map.add_child(layer)
map.add_child(legend)
map.add_child(BindColormap(layer, legend))

#------------------------------------Estudiantes de Nivel Superior por Provincia 2022

layer, legend = makeLayer(gdf_merged, 'Estudiantes de Nivel Superior por Provincia 2022', '2022_Total_Superior', {
    'fields': ['nombre', '2022_Total_Superior', '2022_Porcentaje_Superior', '2022_Porcentaje_Estatal_Superior', '2022_Porcentaje_Privado_Superior'],
    'aliases': ['Provincia:', 'Numero de Estudiantes:', 'Porcentaje de Estudiantes Respecto al Pais:', 'Porcentaje de Estudiantes de Estatales:', 'Porcentaje de Estudiantes de Privadas:']
})

map.add_child(layer)
map.add_child(legend)
map.add_child(BindColormap(layer, legend))

#------------------------------------Estudiantes de Nivel Secundario por Provincia 2022

layer, legend = makeLayer(gdf_merged, 'Estudiantes de Nivel Secundario por Provincia 2022', '2022_Total_Secundario', {
    'fields': ['nombre', '2022_Total_Secundario', '2022_Porcentaje_Secundario', '2022_Porcentaje_Estatal_Secundario', '2022_Porcentaje_Privado_Secundario', '2022_Porcentaje_Repitentes_Secundario'],
    'aliases': ['Provincia:', 'Numero de Estudiantes:', 'Porcentaje de Estudiantes Respecto al Pais:', 'Porcentaje de Estudiantes de Estatales:', 'Porcentaje de Estudiantes de Privadas:', 'Porcentaje de Estudiantes que Repiten:']
})

map.add_child(layer)
map.add_child(legend)
map.add_child(BindColormap(layer, legend))

#------------------------------------Estudiantes de Nivel Primario por Provincia 2022

layer, legend = makeLayer(gdf_merged, 'Estudiantes de Nivel Primario por Provincia 2022', '2022_Total_Primario', {
    'fields': ['nombre', '2022_Total_Primario', '2022_Porcentaje_Primario', '2022_Porcentaje_Estatal_Primario', '2022_Porcentaje_Privado_Primario', '2022_Porcentaje_Repitentes_Primario'],
    'aliases': ['Provincia:', 'Numero de Estudiantes:', 'Porcentaje de Estudiantes Respecto al Pais:', 'Porcentaje de Estudiantes de Estatales:', 'Porcentaje de Estudiantes de Privadas:', 'Porcentaje de Estudiantes que Repiten:']
})

map.add_child(layer)
map.add_child(legend)
map.add_child(BindColormap(layer, legend))

#------------------------------------Estudiantes de Nivel Inicial por Provincia 2022

layer, legend = makeLayer(gdf_merged, 'Estudiantes de Nivel Inicial por Provincia 2022', '2022_Total_Inicial', {
    'fields': ['nombre', '2022_Total_Inicial', '2022_Porcentaje_Inicial', '2022_Porcentaje_Estatal_Inicial', '2022_Porcentaje_Privado_Inicial'],
    'aliases': ['Provincia:', 'Numero de Estudiantes:', 'Porcentaje de Estudiantes Respecto al Pais:', 'Porcentaje de Estudiantes de Estatales:', 'Porcentaje de Estudiantes de Privadas:']
}, True)

map.add_child(layer)
map.add_child(legend)
map.add_child(BindColormap(layer, legend))

#------------------------------------

map.add_child(folium.LayerControl(position='topleft'))

In [38]:
# Descargando el mapa

In [53]:
img_data = map._to_png(5)
img = Image.open(io.BytesIO(img_data))
img.save('image.png')
files.download('image.png')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [41]:
map.save('map.html')
files.download('map.html')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [31]:
######################################################################################################

In [32]:
fig = px.treemap(df_universidades_con_complemento, path=[px.Constant("Argentina"), 'Jurisdicción', 'Institución'], values='2021_Alumnos', custom_data=[
    '2020_Egresados',
    '2021_Nuevos_inscriptos',
    '2021_Reinscriptos',
    '2021_Egresados',
    'Institucion_Universitaria|Universidad',
    'Privada|Provincial|Nacional'
    ], title='N° de Estudiantes de Nivel Universitario por Institucion por Provincia 2021')

fig.update_traces(
    hovertemplate="<b>%{label}</b><br>N° de Estudiantes: %{value}",
    texttemplate="%{label}<br>%{customdata[4]} %{customdata[5]}<br>N° de Egresados 2020: %{customdata[0]}<br>N° de Nuevos Inscriptos 2021: %{customdata[1]}<br>N° de Reinscriptos 2021: %{customdata[2]}<br>N° de Egresados 2021: %{customdata[3]}",
    textposition="middle center"
)

fig.show()

In [33]:
# Descargando el grafico

In [40]:
fig.write_html("figure.html")
files.download('figure.html')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [37]:
fig.write_image("figure.png", scale = 10)
files.download('figure.png')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
######################################################################### Data extra

In [59]:
df.nlargest(3, "2022_Total_Repitentes_Secundario")[['Jurisdicción', '2022_Total_Repitentes_Secundario']]

,Jurisdicción,2022_Total_Repitentes_Secundario
1,Buenos Aires,87814
20,Santa Fe,19503
5,Córdoba,16784


In [61]:
df.nlargest(3, "2022_Total_Repitentes_Primario")[['Jurisdicción', '2022_Total_Repitentes_Primario']]

,Jurisdicción,2022_Total_Repitentes_Primario
1,Buenos Aires,47500
13,Misiones,9070
12,Mendoza,8899


In [66]:
df.nlargest(3, "2021_Total_Universitarios")[['Jurisdicción', '2021_Total_Universitarios']]

,Jurisdicción,2021_Total_Universitarios
1,Buenos Aires,673751
0,Ciudad Autónoma de Buenos Aires,626146
5,Córdoba,344254


In [64]:
df.nlargest(3, "2022_Total_Superior")[['Jurisdicción', '2022_Total_Superior']]

,Jurisdicción,2022_Total_Superior
1,Buenos Aires,261393
0,Ciudad Autónoma de Buenos Aires,124889
5,Córdoba,84346


In [65]:
df.nlargest(3, "2022_Total_Secundario")[['Jurisdicción', '2022_Total_Secundario']]

,Jurisdicción,2022_Total_Secundario
1,Buenos Aires,1677473
5,Córdoba,347517
20,Santa Fe,261165


In [60]:
df.nlargest(3, "2022_Total_Primario")[['Jurisdicción', '2022_Total_Primario']]

,Jurisdicción,2022_Total_Primario
1,Buenos Aires,1734718
20,Santa Fe,388054
5,Córdoba,357084


In [62]:
df.nlargest(3, "2022_Total_Inicial")[['Jurisdicción', '2022_Total_Inicial']]

,Jurisdicción,2022_Total_Inicial
1,Buenos Aires,672694
5,Córdoba,136445
20,Santa Fe,118501


In [77]:
# EL grado de diferencia de los alumnos de inicial contra los de primaria
df_tmp_a = df[['Jurisdicción', '2022_Total_Inicial', '2022_Total_Primario']]
df_tmp_a['grado_de_diferencia'] = ((df['2022_Total_Primario']/df['2022_Total_Inicial'])).round(2)
df_tmp_a

/tmp/ipython-input-77-3496788043.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Jurisdicción,2022_Total_Inicial,2022_Total_Primario,grado_de_diferencia
0,Ciudad Autónoma de Buenos Aires,105905,278233,2.63
1,Buenos Aires,672694,1734718,2.58
2,Catamarca,15693,42657,2.72
3,Chaco,51422,170078,3.31
4,Chubut,25166,60239,2.39
5,Córdoba,136445,357084,2.62
6,Corrientes,44214,130195,2.94
7,Entre Ríos,51527,142480,2.77
8,Formosa,20281,71956,3.55
9,Jujuy,29022,93931,3.24


In [86]:
# El porcentaje de alumnos de las mas grandes 3 universidades contra el total de alumnos
((df_universidades_con_complemento.nlargest(3, "2021_Alumnos")[['2021_Alumnos']].sum() / df_universidades_con_complemento['2021_Alumnos'].sum())*100).round(2)

,0
2021_Alumnos,26.24


In [89]:
# numero de estudiantes 2021
df_universidades_con_complemento['2021_Alumnos'].sum()

np.int64(2549789)

In [90]:
df_universidades_con_complemento.nlargest(3, "2021_Alumnos")[['Jurisdicción', '2021_Alumnos']].sum()

,0
Jurisdicción,Ciudad Autónoma de Buenos AiresCórdobaBuenos A...
2021_Alumnos,669158


In [ ]:
# porcentaje de estudiantes estatal vs privado niveles: universitario, superior, secundario, primario, inicial

In [93]:
df[['Jurisdicción', '2021_Porcentaje_Estatal_Universitarios', '2021_Porcentaje_Privado_Universitarios', '2022_Porcentaje_Estatal_Superior', '2022_Porcentaje_Privado_Superior',
    '2022_Porcentaje_Estatal_Secundario', '2022_Porcentaje_Privado_Secundario', '2022_Porcentaje_Estatal_Primario', '2022_Porcentaje_Privado_Primario',
    '2022_Porcentaje_Estatal_Inicial', '2022_Porcentaje_Privado_Inicial']]

,Jurisdicción,2021_Porcentaje_Estatal_Universitarios,2021_Porcentaje_Privado_Universitarios,2022_Porcentaje_Estatal_Superior,2022_Porcentaje_Privado_Superior,2022_Porcentaje_Estatal_Secundario,2022_Porcentaje_Privado_Secundario,2022_Porcentaje_Estatal_Primario,2022_Porcentaje_Privado_Primario,2022_Porcentaje_Estatal_Inicial,2022_Porcentaje_Privado_Inicial
0,Ciudad Autónoma de Buenos Aires,70.0%,30.0%,42.95%,57.05%,50.7%,49.3%,52.3%,47.7%,48.25%,51.75%
1,Buenos Aires,91.87%,8.13%,80.81%,19.19%,69.49%,30.51%,67.11%,32.89%,65.64%,34.36%
2,Catamarca,100.0%,0.0%,85.79%,14.21%,80.38%,19.62%,79.14%,20.86%,74.79%,25.21%
3,Chaco,96.59%,3.41%,67.21%,32.79%,81.7%,18.3%,86.88%,13.12%,82.22%,17.78%
4,Chubut,100.0%,0.0%,82.7%,17.3%,85.48%,14.52%,84.47%,15.53%,81.21%,18.79%
5,Córdoba,70.25%,29.75%,62.08%,37.92%,60.55%,39.45%,72.61%,27.39%,73.23%,26.77%
6,Corrientes,75.44%,24.56%,85.84%,14.16%,83.07%,16.93%,87.08%,12.92%,81.46%,18.54%
7,Entre Ríos,89.37%,10.63%,73.25%,26.75%,74.95%,25.05%,72.4%,27.6%,69.24%,30.76%
8,Formosa,100.0%,0.0%,86.06%,13.94%,89.88%,10.12%,90.3%,9.7%,76.18%,23.82%
9,Jujuy,91.43%,8.57%,68.4%,31.6%,84.28%,15.72%,87.23%,12.77%,77.57%,22.43%


In [96]:
#------------------------------------Haciendo un dataframe para mostrar la relacion entre educacion estatal vs privada por nivel en word
Total_Universitarios = df['2021_Total_Universitarios'].sum()
Total_Superior = df['2022_Total_Superior'].sum()
Total_Secundario = df['2022_Total_Secundario'].sum()
Total_Primario = df['2022_Total_Primario'].sum()
Total_Inicial = df['2022_Total_Inicial'].sum()

df_tmp = pd.DataFrame({
  'Nivel': ['Inicial', 'Primario', 'Secundario', 'Superior', 'Universitario'],
  'Porcion Estatal': [
      ((df['2022_Estatal_Inicial'].sum() * 100) / Total_Inicial).round(2).astype(str) + '%',
      ((df['2022_Estatal_Primario'].sum() * 100) / Total_Primario).round(2).astype(str) + '%',
      ((df['2022_Estatal_Secundario'].sum() * 100) / Total_Secundario).round(2).astype(str) + '%',
      ((df['2022_Estatal_Superior'].sum() * 100) / Total_Superior).round(2).astype(str) + '%',
      ((df['2021_Estatal_Universitarios'].sum() * 100) / Total_Universitarios).round(2).astype(str) + '%'
  ],
  'Porcion Privada': [
      ((df['2022_Privado_Inicial'].sum()  * 100) / Total_Inicial).round(2).astype(str) + '%',
      ((df['2022_Privado_Primario'].sum()  * 100) / Total_Primario).round(2).astype(str) + '%',
      ((df['2022_Privado_Secundario'].sum()  * 100) / Total_Secundario).round(2).astype(str) + '%',
      ((df['2022_Privado_Superior'].sum()  * 100) / Total_Superior).round(2).astype(str) + '%',
      ((df['2021_Privado_Universitarios'].sum()  * 100) / Total_Universitarios).round(2).astype(str) + '%'
  ]
})

display(df_tmp)




,Nivel,Porcion Estatal,Porcion Privada
0,Inicial,69.92%,30.08%
1,Primario,74.03%,25.97%
2,Secundario,71.73%,28.27%
3,Superior,68.82%,31.18%
4,Universitario,80.99%,19.01%
